<a href="https://colab.research.google.com/github/talkin24/GradientBoosting/blob/main/Ch06_XGB_Hyperparameters_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/GradientBoosting

Mounted at /content/drive
/content/drive/MyDrive/GradientBoosting


In [3]:
 import pandas as pd

 df = pd.read_csv('./heart_disease.csv')
 df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [16]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [17]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')

In [8]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(model, X, y, cv=5)
print('accuracy:', scores)
print('accuracy mean:', scores.mean())

accuracy: [0.85245902 0.85245902 0.7704918  0.78333333 0.76666667]
accuracy mean: 0.8050819672131148


In [9]:
from sklearn.model_selection import StratifiedKFold

In [18]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [11]:
print('전체 데이터 클래스 분포:', np.bincount(y))
print()
for split_no, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
  print('{}번째 훈련 폴드:'.format(split_no+1), np.bincount(y[train_idx]))
  print('{}번째 검증 폴드:'.format(split_no+1), np.bincount(y[test_idx]))
  print()

전체 데이터 클래스 분포: [138 165]

1번째 훈련 폴드: [110 132]
1번째 검증 폴드: [28 33]

2번째 훈련 폴드: [110 132]
2번째 검증 폴드: [28 33]

3번째 훈련 폴드: [110 132]
3번째 검증 폴드: [28 33]

4번째 훈련 폴드: [111 132]
4번째 검증 폴드: [27 33]

5번째 훈련 폴드: [111 132]
5번째 검증 폴드: [27 33]



In [13]:
kfold

StratifiedKFold(n_splits=5, random_state=2, shuffle=True)

In [19]:
scores = cross_val_score(model, X, y, cv=kfold)
print('accuracy:', np.round(scores, 2))
print('accuracy mean:', np.round(scores.mean(), 2))

accuracy: [0.75 0.75 0.75 0.82 0.87]
accuracy mean: 0.79


In [20]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [23]:
def grid_search(params, random=False):
  xgb = XGBClassifier(booster='gbtree', objective='binary:logistic',
                      random_state=2, verbosity=0, use_label_encoder=False)
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

  if random:
    grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iters=20,
                              n_jobs=-1, random_state=2)
  else:
    grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
  
  grid.fit(X, y)
  best_params = grid.best_params_
  print('best parameters:', best_params)
  best_score = grid.best_score_
  print('best score:', best_score)

In [24]:
grid_search(params={'n_estimators':[100, 200, 400, 800]})

best parameters: {'n_estimators': 100}
best score: 0.7891256830601093


In [25]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

best parameters: {'learning_rate': 0.05}
best score: 0.8056284153005464


In [26]:
grid_search(params={'max_depth':[2, 3, 5, 6, 8]})

best parameters: {'max_depth': 2}
best score: 0.8056830601092896


In [27]:
grid_search(params={'gamma':[0, 0.01, 0.1, 0.5, 1, 2]})

best parameters: {'gamma': 1}
best score: 0.8022404371584699


In [28]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5]})

best parameters: {'min_child_weight': 2}
best score: 0.8087978142076502


In [29]:
grid_search(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

best parameters: {'subsample': 0.7}
best score: 0.8056830601092896


In [30]:
grid_search(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

best parameters: {'colsample_bytree': 0.5}
best score: 0.7989071038251366
